# Ejercicios Pair

In [33]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
#import scipy.stats as stats
import scipy.stats as stats
from scipy.stats import shapiro, kstest

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames 

#para desplegar todos los decimales y no tener el formato científico:
#👀 OJO! estar atenta cuando lo pones que no estás perdiendo información
pd.options.display.float_format = '{:.2f}'.format 

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

# Ejercicios ETL Parte I

En este caso trabajas en una empresa de venta al por menor de productos italianos y debes realizar la limpieza, transformación e integración de datos de ventas, productos y clientes para su análisis.

Los pasos que deberás seguir en este ejercicio son:

## 1. Lectura de la Información:

Leer los archivos CSV (ventas.csv, productos.csv, clientes.csv).

Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

In [34]:
df_ventas = pd.read_csv('ventas.csv', index_col = 0)
df_ventas.head()

,ID_Producto,Fecha_Venta,Cantidad,Total
ID_Cliente,,,,
723,A1,2023-11-22,2,17.98
498,C3,2023-11-21,1,5.49
121,D4,2023-11-20,3,32.97
885,L12,2023-11-19,1,6.49
347,Q17,2023-11-18,2,7.98


In [35]:
df_clientes = pd.read_csv('clientes.csv', index_col = 0)
df_clientes.head()

,first_name,last_name,email,gender,City,Country,Address
id,,,,,,,
1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle


In [36]:
df_pctos = pd.read_csv('productos.csv', index_col = 0)
df_pctos.sample()

,Nombre_Producto,Categoria,Precio,Origen,Descripcion,Des1,Des2,Des3,Des4
ID,,,,,,,,,
CC29,Carbonara,Platos Preparados,8.75,Italia,Pasta con salsa de huevo,queso,panceta y pimienta negra,una receta clásica de Roma.,NaN


In [37]:
df_pctos['Des_completa'] = (
    df_pctos['Descripcion'] + "," +
    df_pctos['Des1'].fillna('') + "," +
    df_pctos['Des2'].fillna('') + "," +
    df_pctos['Des3'].fillna('') + "," +
    df_pctos['Des4'].fillna('')
)

In [38]:
df_pctos = df_pctos.drop(columns = ['Descripcion', 'Des1', 'Des2', 'Des3', 'Des4'])

In [39]:
df_pctos.head()

,Nombre_Producto,Categoria,Precio,Origen,Des_completa
ID,,,,,
A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos, una d..."
C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...
E5,Orecchiette,Productos Secos,4.29,Italia,Pequeñas pastas con forma de orecchiette ideal...



## 2. Transformación de Datos:

- Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.

- Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

- Aplicar transformaciones relevantes según sea necesario: por ejemplo, convertir tipos de datos, renombrar columnas, crear nuevas características derivadas, etc.

In [40]:
def comprobar_nulos(dic_df):

    for nombre_df, df in dic_df.items():

        print(nombre_df)
        print(df.isnull().sum()/df.shape[0]*100)
        print('-' *20)

In [41]:
diccionario_df = {'clientes': df_clientes, 'productos': df_pctos, 'ventas': df_ventas}

In [42]:
comprobar_nulos(diccionario_df)

clientes
first_name    0.00
last_name     0.00
email         2.70
gender        7.70
City         12.40
Country      15.40
Address       4.10
dtype: float64
--------------------
productos
Nombre_Producto   0.00
Categoria         0.00
Precio            0.00
Origen            0.00
Des_completa      0.00
dtype: float64
--------------------
ventas
ID_Producto   0.00
Fecha_Venta   0.00
Cantidad      0.00
Total         0.00
dtype: float64
--------------------


In [43]:
df_clientes.head()

,first_name,last_name,email,gender,City,Country,Address
id,,,,,,,
1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing
4,Filmer,Eirwin,feirwin3@intel.com,NaN,Leon,Spain,5 American Ash Road
5,Tanhya,Lubbock,tlubbock4@huffingtonpost.com,Female,"Hospitalet De Llobregat, L'",Spain,9289 Merry Circle


In [44]:
def comprobar_duplicados(dic_df):
    
    for nombre_df, df in dic_df.items():

        print(nombre_df)
        print(df.duplicated().sum())
        print('-' *20)

In [45]:
comprobar_duplicados(diccionario_df)

clientes
0
--------------------
productos
0
--------------------
ventas
0
--------------------


- Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

In [46]:
df_ventas.head(3)

,ID_Producto,Fecha_Venta,Cantidad,Total
ID_Cliente,,,,
723,A1,2023-11-22,2,17.98
498,C3,2023-11-21,1,5.49
121,D4,2023-11-20,3,32.97


In [47]:
df_clientes.head(3)

,first_name,last_name,email,gender,City,Country,Address
id,,,,,,,
1,Cheri,Dunsmore,cdunsmore0@instagram.com,Female,Palma De Mallorca,Spain,076 Rockefeller Crossing
2,Hunt,Bartomeu,hbartomeu1@nsw.gov.au,Male,Lugo,Spain,0046 Utah Junction
3,Michaeline,Paynton,mpaynton2@narod.ru,Female,NaN,Spain,0 Corry Crossing


In [48]:
df_pctos.head(3)

,Nombre_Producto,Categoria,Precio,Origen,Des_completa
ID,,,,,
A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos, una d..."
C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...


In [49]:
df_completo = df_ventas.join(df_clientes, how = "left")

In [50]:
df_completo.head(3)

,ID_Producto,Fecha_Venta,Cantidad,Total,first_name,last_name,email,gender,City,Country,Address
ID_Cliente,,,,,,,,,,,
723,A1,2023-11-22,2,17.98,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road
498,C3,2023-11-21,1,5.49,Padgett,Painter,ppainterdt@scientificamerican.com,Male,Barcelona,Spain,5 Artisan Lane
121,D4,2023-11-20,3,32.97,Wye,Sinncock,wsinncock3c@fc2.com,Male,Santander,NaN,4679 Fair Oaks Trail


In [68]:
df_completo.reset_index(drop= False, inplace=True) 

In [70]:
df_completo.head(2)

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total,first_name,last_name,email,gender,City,Country,Address
0,723,A1,2023-11-22,2,17.98,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road
1,498,C3,2023-11-21,1,5.49,Padgett,Painter,ppainterdt@scientificamerican.com,Male,Barcelona,Spain,5 Artisan Lane


In [71]:
df_pctos.shape

(33, 6)

In [72]:
df_pctos.reset_index(drop= False, inplace=True) # cambiamos la columna ID sin eliminarla

In [73]:
df_pctos.head(2)

,index,ID,Nombre_Producto,Categoria,Precio,Origen,Des_completa
0,0,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
1,1,B2,Risotto de Champiñones,Platos Preparados,6.75,Italia,"Risotto cremoso con champiñones frescos, una d..."


In [74]:
df_total = df_completo.merge( right=df_pctos, how = "left", left_on="ID_Producto", right_on="ID")

In [75]:
df_total.head(3)

,ID_Cliente,ID_Producto,Fecha_Venta,Cantidad,Total,first_name,last_name,email,gender,City,Country,Address,index,ID,Nombre_Producto,Categoria,Precio,Origen,Des_completa
0,723,A1,2023-11-22,2,17.98,Sephira,Bleackley,sbleackleyk2@liveinternet.ru,NaN,Girona,NaN,4 Birchwood Road,0.00,A1,Pizza Margherita,Platos Preparados,8.99,Italia,"Clásica pizza italiana con tomate, mozzarella ..."
1,498,C3,2023-11-21,1,5.49,Padgett,Painter,ppainterdt@scientificamerican.com,Male,Barcelona,Spain,5 Artisan Lane,2.00,C3,Tiramisú,Postres,5.49,Italia,Postre clásico italiano con capas de bizcocho ...
2,121,D4,2023-11-20,3,32.97,Wye,Sinncock,wsinncock3c@fc2.com,Male,Santander,NaN,4679 Fair Oaks Trail,3.00,D4,Panettone,Repostería,10.99,Italia,Pan dulce navideño italiano con frutas confita...


In [76]:
df_total.shape

(100, 19)